# Modeler

## TextDataの読み込み
- 分かち書きされたテキストデータが必要
-  (現在local運営)
### Google Drive
- Google Driveに保存してあるデータを引っ張ってくる
- 認証(マウント)が必要

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


- またはアップロードする(重いときついか...?)

## Model作成準備

- gensimは入っているのでOK
- 適宜importのパスを見ておく

## Optionを設定する
### sentences
解析に使う1行1センテンスで書かれた文書。日本語の場合はLineSentenceフォーマットを使えばうまくいった。単語が空白文字で区切られていて、文章は改行で区切られていれば問題ない。
### sg
{1,0}の整数で訓練アルゴリズムを設定できる。 1を選べばskip-gramで、0ならばCBOWを使う。
### size
特徴ベクトルの次元を設定する。
### window
文書内における現在の単語と予測した単語の間の距離の最大値を設定する。言い換えると、文脈の最大単語数を設定する。
### alpha
学習率の初期値を設定する。
### min_alpha
訓練の過程で徐々に落ちていく学習率の最小値を設定する。
### seed
乱数を生成する際のシード番号を設定する。
### min_count
一定の頻度以下の単語を除外する際の値を設定する。
### max_vocab_size
語彙ベクトルを構築している際のメモリ制限を設定する。
### sample
(0, 1e-5)の範囲で、頻度語がランダムに削除される閾値を設定する。高速化と精度向上を狙っており、自然言語処理においても高頻度語はストップワードとして除去するなどの対応が取られている。
### workers
モデルを訓練するために多くのワーカースレッドを利用するかどうか設定する。（並列化関連）
### hs
{1,0}の整数で、1であれば階層的ソフトマックスがモデルの訓練で用いられ、0であり引数negativeがnon-zeroであればネガティヴサンプリングが設定できる。全部計算することが大変なので、階層的なグループに分けて各グループごとに学習するというのがモチベーション。
### negative
0よりも大きければネガティブサンプリングが用いられる。5〜20などを選び、どれだけノイズワードが描かれているかを識別する。0であればネガティブサンプリングが適用されない。ネガティブサンプリングは計算高速化を目的に出力層で正解ニューロン以外のニューロンを更新しないように学習する手法。
### cbow_mean
{1,0}の整数で、0であれば単語ベクトルの合計を用い、1であればCBOWが用いられた際の平均が用いられる。
### hashfxn
訓練の再現性のためにランダムに初期値のウエイト付けできる。
### iter
コーパスにおける繰り返し回数（エポック数）を設定できる。
### trim_rule
ある単語を語彙に含めるべきかどうかを識別する、語彙のトリミングルールを設定する。
### sorted_vocab
{1,0}の整数で、1であれば頻度の降順で語彙を並べ替える。
### batch_words
ワーカースレッドにわたすバッチの大きさを指定する。（並列化関連）
### compute_loss
Trueであれば損失関数の計算と蓄積を行う。
### callbacks
訓練時の特定の段階で実行する際に必要なコールバックのリストを指定できる。

In [2]:
from gensim.models import word2vec
import logging

# logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                        level=logging.INFO)

print("読み込みファイルパスを入力")
import_path = input()
sentences = word2vec.Text8Corpus(import_path)

# 先にしておく
print("保存先を入力")
save_path = input()

model = word2vec.Word2Vec(
                                  sentences,
                                  sg=1, # skipgram
                                  size=10000,
                                  min_count=10, 
                                  window=5,
                                  hs=1,
                                  batch_words=50)

model.save(save_path)


読み込みファイルパスを入力
保存先を入力


2019-05-22 11:52:14,154 : INFO : collecting all words and their counts
2019-05-22 11:52:14,155 : WARNING : this function is deprecated, use smart_open.open instead
2019-05-22 11:52:14,685 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-05-22 11:52:16,379 : INFO : collected 38258 word types from a corpus of 5912864 raw words and 592 sentences
2019-05-22 11:52:16,380 : INFO : Loading a fresh vocabulary
2019-05-22 11:52:16,521 : INFO : effective_min_count=10 retains 15809 unique words (41% of original 38258, drops 22449)
2019-05-22 11:52:16,523 : INFO : effective_min_count=10 leaves 5841765 word corpus (98% of original 5912864, drops 71099)
2019-05-22 11:52:16,569 : INFO : deleting the raw counts dictionary of 38258 items
2019-05-22 11:52:16,573 : INFO : sample=0.001 downsamples 37 most-common words
2019-05-22 11:52:16,574 : INFO : downsampling leaves estimated 5008380 word corpus (85.7% of prior 5841765)
2019-05-22 11:52:16,595 : INFO : constructing a huff